In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import rasterio as rio
import folium
import cv2 as cv

In [2]:
#lat=18.200178; lon=-66.664513
lat=18.200178; lon=-66.064513
def overlay_image_on_puerto_rico(data, band_layer=2):
    band = data
    #band = np.array(list(map(lambda x: np.array(list(map(lambda y: 0.0 if np.isnan(y) else y, x))), band))) #puts nan to 0
    #band += np.abs(band.min()) # Added to account for negative values. Might have to normalize the picture.
    m = folium.Map([lat, lon], zoom_start=9.45)
    folium.raster_layers.ImageOverlay(
        image=band,
        bounds = [[18.6,-67.3,],[17.9,-65.2]],
        colormap=lambda x: (1, 0, 0, x),
    ).add_to(m)
    return m

def readFile(name):
    #p = './eie_data/'
    image= rio.open(name)
    return(image)

def inpaintImg(im):
    row, col = im.shape
    avg_val = np.average(im)
    mask = np.ones((row+2,col+2))
    mask[1:-1,1:-1] = np.isnan(im)
    mask = np.uint8(mask)
    new_im = np.ones((row+2,col+2))*avg_val
    new_im[1:-1,1:-1] = im
    new_im = np.float32(new_im)
    new_im = np.float64(cv.inpaint(new_im, mask, 3, cv.INPAINT_NS)) # Inpaints using Navier-Stokes 
    a = im.clip(min=0) # sets negative values to zero
    im = new_im[1:-1,1:-1]
    return im

In [3]:
import pathlib
def dataLoader(folder):
    #p = "./ds4g-environmental-insights-explorer/eie_data/"
    p = "./eie_data/"
    for path in pathlib.Path(p+folder+"/").iterdir():
        if path.is_file():
            image= rio.open(path)
            band2 = image.read(2)
            if np.sum(np.isnan(band2)) <= image.shape[0]*image.shape[1]*0.05:
                yield(image)
            
loader_s5p = dataLoader("s5p_no2")
#loader_gfs = dataLoader("gfs")
#loader_gldas = dataLoader("gldas")

We first calculate various statistics:

In [4]:
pic = readFile('./eie_data/s5p_no2/s5p_no2_20190504T165553_20190510T183608.tif')
pic.name.split("/")[-1][:-4]

's5p_no2_20190504T165553_20190510T183608'

## Test dataloader

In [5]:
loader_s5p = dataLoader("s5p_no2")

In [6]:
# Creates NA-map
#img = next(loader_s5p)
img = readFile("eie_data\cleaned_s5p_no2\cleaned_s5p_no2_20180712T160623_20180719T105633.tif")
#img = readFile("eie_data\s5p_no2\s5p_no2_20180706T161914_20180712T200737.tif")
print(img.name)
im = img.read(2)
lst = []
for i, item1 in enumerate(im):
    for j, item2 in enumerate(item1):
        if np.isnan(item2):
            lst.append([i,j])
lst = np.array(lst)
print("len list: ", len(lst))
print("NA Count: ", np.sum(np.isnan(im)))

eie_data\cleaned_s5p_no2\cleaned_s5p_no2_20180712T160623_20180719T105633.tif
len list:  0
NA Count:  0


In [7]:
if len(lst) > 0:
    plt.subplot(1, 2, 1)
    plt.scatter(lst[:,0], lst[:,1], s=0.5)

In [8]:
overlay_image_on_puerto_rico(inpaintImg(im))

## Clean Data

In [9]:
# Only 128 images does not contain NA-values in the second band.
# If we accept all pictures with less than 5% NA values we get 324 images.
# A total of 387 images are availible.

In [9]:
# Save names of all valid pictures in order:
loader_s5p = dataLoader("s5p_no2")
s5p_order = []
for img in loader_s5p:
    date = img.name.split("\\")[-1].split("_")[-2][:8]
    s5p_order.append([date, img.name])
# Date ordered:
s5p_order.sort(key=(lambda x: x[0]))   
# Number of valid pictures:
print("Number of availible images: ", len(s5p_order))

Number of availible images:  324


In [10]:
# Write images to new folder:
def writeCleanData(s5p_order):
    for i, [date, name] in enumerate(s5p_order):
        pic = readFile(name)
        new_dataset = rio.open(name.replace("s5p_no2", "cleaned_s5p_no2"),
                               "w",
                               driver=pic.meta['driver'],
                               dtype=pic.meta['dtype'],
                               height=pic.meta['height'],
                               width=pic.meta['width'],
                               count=pic.meta['count'],
                               crs=pic.meta['crs'],
                               transform=pic.meta['transform'])
        band =pic.read(2)
        band = inpaintImg(band)
        new_dataset.write(band, 2)
        new_dataset.close()
    return
#writeCleanData(s5p_order)

In [11]:
s5p_order_c = s5p_order
for i, [date, name] in enumerate(s5p_order_c):
    s5p_order_c[i] = [date, name.replace("s5p_no2", "cleaned_s5p_no2")]

## Visualize NO2 on Puerto Rico

In [15]:
def loadValid(lst):
    for i, row in enumerate(lst):
        image= rio.open(row[1])
        yield(image)
def loadDiff(lst):
    for i, row in enumerate(lst):
        if i!=0:
            image1= rio.open(lst[i-1][1])
            image2= rio.open(lst[i][1])
            img=image2.read(2)-image1.read(2)
            yield(img)

s5p_clean_loader = loadValid(s5p_order_c)
#s5p_inorder_diff = loadDiff(s5p_order)

In [16]:
import selenium.webdriver

driver = selenium.webdriver.PhantomJS()
driver.set_window_size(950, 380)  # choose a resolution

for frame in s5p_clean_loader:
    m = overlay_image_on_puerto_rico(frame.read(2))
    m.save('temp_map.html')
    driver.get('temp_map.html')
    p= "./eie_data/cleaned_s5p_no2_overlays/"
    driver.save_screenshot(p + frame.name.split("\\")[-1][:-4] + '.png')
    print(frame.name.split("\\")[-1][:-4])


C:\Users\aabil\Anaconda3\lib\site-packages\selenium\webdriver\phantomjs\webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


cleaned_s5p_no2_20180701T161259_20180707T175356
cleaned_s5p_no2_20180702T173526_20180708T192358
cleaned_s5p_no2_20180704T165720_20180710T184641
cleaned_s5p_no2_20180705T163817_20180712T105412
cleaned_s5p_no2_20180706T161914_20180712T200737
cleaned_s5p_no2_20180707T174140_20180713T191854
cleaned_s5p_no2_20180708T172237_20180714T190743
cleaned_s5p_no2_20180710T164430_20180716T182411
cleaned_s5p_no2_20180711T162527_20180718T185658
cleaned_s5p_no2_20180712T160623_20180719T105633
cleaned_s5p_no2_20180713T172849_20180719T201407
cleaned_s5p_no2_20180714T170945_20180720T185244
cleaned_s5p_no2_20180715T165041_20180721T182929
cleaned_s5p_no2_20180716T163137_20180722T180949
cleaned_s5p_no2_20180717T161233_20180723T175248
cleaned_s5p_no2_20180718T173458_20180724T193016
cleaned_s5p_no2_20180719T171554_20180725T185631
cleaned_s5p_no2_20180720T165649_20180726T182222
cleaned_s5p_no2_20180721T163745_20180727T180656
cleaned_s5p_no2_20180722T161840_20180728T175208
cleaned_s5p_no2_20180723T174105_20180729

cleaned_s5p_no2_20190118T163240_20190124T183944
cleaned_s5p_no2_20190119T161341_20190125T175413
cleaned_s5p_no2_20190120T155443_20190126T173142
cleaned_s5p_no2_20190121T171715_20190127T191156
cleaned_s5p_no2_20190122T165817_20190128T190949
cleaned_s5p_no2_20190123T163919_20190129T184821
cleaned_s5p_no2_20190124T162022_20190130T182515
cleaned_s5p_no2_20190125T160124_20190131T175843
cleaned_s5p_no2_20190126T172357_20190201T192405
cleaned_s5p_no2_20190127T170459_20190202T185712
cleaned_s5p_no2_20190128T164602_20190203T184133
cleaned_s5p_no2_20190129T162704_20190204T181439
cleaned_s5p_no2_20190130T160807_20190205T180152
cleaned_s5p_no2_20190201T171143_20190207T185559
cleaned_s5p_no2_20190202T165246_20190208T185635
cleaned_s5p_no2_20190203T163350_20190209T183011
cleaned_s5p_no2_20190204T161453_20190210T180935
cleaned_s5p_no2_20190205T155556_20190211T174416
cleaned_s5p_no2_20190206T171830_20190212T190232
cleaned_s5p_no2_20190207T165933_20190213T190304
cleaned_s5p_no2_20190208T164037_20190214

In [17]:
def readOverlay(lst):
    im_lst = []
    p= "./eie_data/cleaned_s5p_no2_overlays/"
    for i, (_, name) in enumerate(lst):
        im = cv.imread(p + name.split("\\")[-1][:-4] + '.png')
        im_lst.append(im)
    np_im_lst = np.array(im_lst)
    return(np_im_lst)
overlay_loader = readOverlay(s5p_order_c)

## Show overlay film

In [20]:
def showOverlayFilm(loader):
    %matplotlib tk
    fig, ax = plt.subplots(1,1)
    for frame in loader:
        ax.cla()
        ax.imshow(frame)
        plt.pause(0.2)
    plt.close()
    %matplotlib inline
    return
showOverlayFilm(overlay_loader)

## Extra stuff

In [14]:
#pic_sum = np.zeros((148, 475))
#c=1
#for i in s5p_inorder:
#    if c%14==0:
#        pic_sum = pic_sum = np.zeros((148, 475))
#        c=1
#        pic_sum = i.read(2)
#        print('reset')
#    else:
#        pic_sum += i.read(2)
#        c+=1
#pic_sum = pic_sum/7
#overlay_image_on_puerto_rico(next(s5p_inorder).read(2),band_layer=2)